In [1]:
import sys
sys.path.insert(0, '/Users/pllueca/code/pyglitch')

In [2]:
import imageio
import sh
import ffmpeg
import subprocess

In [6]:
import numpy as np

from glitch import (
    swap_block,
    salt_and_pepper,
    move_channels_random,
    move_channel,
    move_random_blocks,
    get_video_size,
    start_ffmpeg_writer,
    start_ffmpeg_reader,
    read_frame
)

In [40]:
in_fname = 'IMG_7447.mov'
out_fname = 'IMG_7447_glitch.mp4'
width, height = get_video_size(in_fname)
reader = start_ffmpeg_reader(in_fname)
writer = start_ffmpeg_writer(out_fname, width, height)
frame_idx = 0

# each glitch (effect) happens during some frames
min_effect_length = 1
max_effect_length = 15
remaining_frames_effect = 0
while True:
    if frame_idx % 100 == 0:
        print(f'frame {frame_idx}')
        
    frame = read_frame(reader, width, height)
    if frame is None:
        # no more frames
        break
    if not remaining_frames_effect:
        remaining_frames_effect = np.random.randint(min_effect_length, max_effect_length)
        current_effect_frame = 1
        # roll for next effect: noise and block swapping
        roll = np.random.randint(0,7)
        if frame_idx < 5:
            roll = 4
#         roll = 0
        
        # 0 -> move channels progresively
        if roll in [0, 5]:
            channel_directions = np.random.randint(-3,3,(3, 2))
            remaining_frames_effect = 20

        # 1 -> "vibrate channels"
        if roll in [1]:
            remaining_frames_effect = 5
        # 2 -> swap blocks static
        if roll in [2]:
            num_blocks = np.random.randint(1, 4)
            block_sizes = np.random.randint(
                100, 600,
                (num_blocks, 2)
            )
            block_channels = np.random.randint(0, 3, (num_blocks,))
            block_xs, block_ys = [], []
            for b in range(num_blocks):
                block_xs.append(np.random.randint(0, height-block_sizes[b,0], (2,)))
                block_ys.append(np.random.randint(0, width-block_sizes[b,1], (2,)))
            block_xs = np.asarray(block_xs)
            block_ys = np.asarray(block_ys)

        # 3 -> swap blocks random
        # 5 -> channels and blocks
        # 4+ -> nothing

        roll_noise = np.random.randint(0,3)
        # if 0 or 1 noise
    else:
        remaining_frames_effect -= 1
        current_effect_frame += 1
    frame_orig = frame
    frame = frame.copy()
    
    if roll in [0, 5]:
        for c in range(3):
            dx, dy = channel_directions[c] * current_effect_frame
            frame = move_channel(frame, c, dx, dy)        
    
    if roll in [1]:
        frame = move_channels_random(frame, -15, 15)
    

    if roll in [3, 5]:
        num_blocks = np.random.randint(1, 5)
        block_sizes = np.random.randint(
            50, 400,
            (num_blocks, 2)
        )
        block_channels = np.random.randint(0, 3, (num_blocks,))
        block_xs, block_ys = [], []
        for b in range(num_blocks):
            block_xs.append(np.random.randint(0, height-block_sizes[b,0], (2,)))
            block_ys.append(np.random.randint(0, width-block_sizes[b,1], (2,)))
        block_xs = np.asarray(block_xs)
        block_ys = np.asarray(block_ys)
        
    if roll in [2, 3, 5]:
        for b in range(num_blocks):
            origin_x, dst_x = block_xs[b]
            origin_y, dst_y = block_ys[b]            
            swap_block(
                frame_orig,
                frame,
                origin_x, origin_y,
                dst_x, dst_y,
                block_sizes[b][0],
                block_sizes[b][1],
                block_channels[b]
            )
            
    if roll_noise in [0, 1]:
        frame = salt_and_pepper(frame,0.5, .02)

    writer.stdin.write(frame.astype(np.uint8).tobytes())
    frame_idx += 1

# cleanup
reader.wait()
writer.stdin.close()
writer.wait()
print('done')

frame 0
frame 100
frame 200
frame 300
done
